In [ ]:
from rnn.model import *
from rnn.train import *
from rnn.predict import *
from utils.data_loader import *

In [ ]:
# Hyper params
hidden_size = 128
batch_size = 10
sequence_len = 200
lr = 0.01
n_layers = 1
n_epochs = -1
print_cycle = 100
save_cycle = 10000

# Data path
rawdata_dir = 'raw_data'
middata_dir = 'mid_data'
poem_dir = "poem_result"
model_dir = "rnn_models"

In [ ]:
# loda data with preprocessing
loader = data_loader(sequence_len, batch_size, rawdata_dir, middata_dir)
loader.preprocess()

In [ ]:
# load data from file
# data loader
loader = data_loader(sequence_len, batch_size, rawdata_dir, middata_dir)
loader.load()

In [ ]:
# load functionality check
cb = loader.cb
data_batch = loader.np_batchs
print("Input Vector Size : " + str(cb.size()))
print("Total Batch Num : " + str(len(data_batch)))
print(data_batch)

In [ ]:
# Required Params
input_size = cb.size()
output_size = cb.size()
start_sequence = cb.get_number("A")
# Model
decoder = RNN(input_size, hidden_size, output_size, n_layers)
optimizer = torch.optim.Adam(decoder.parameters(), lr)
criterion = nn.CrossEntropyLoss()
start = time.time()
all_losses = []
loss_avg = 0

model_num = 0

#print(*random_training_set(data_batch, sequence_len, batch_size))

try:
    if (n_epochs == -1):
        epoch = 0
        while (True):
            epoch += 1
            loss = train(decoder, optimizer, criterion, sequence_len, batch_size , *random_training_set(data_batch, sequence_len, batch_size))
            loss_avg += loss
            
            if epoch % print_cycle == 0:
                print('[Time : %s, Epoach : (%d), Loss : %.4f]' % (time_since(start), epoch, loss))
                
            if (epoch % save_cycle == 0):
                model_num += 1
                print("Save %d model" % (model_num))
                f = open("./" + poem_dir + "/poem" + str(model_num) + ".txt", 'w')
                predicted = predict(decoder, start_sequence, 1000)
                f.write(cb.get_string(predicted))
                save(decoder, "./"+model_dir+"/rnn_model"+str(model_num))
                f.close()

    else :
        # TODO
        print("hi")
        
except KeyboardInterrupt:
    print("Saving before quit...")
    save(decoder, "./"+model_dir+"/rnn_model"+str(model_num))